In [ ]:
import os
import pandas as pd
import numpy as np
from numpy import sort
from os import listdir
from os.path import isfile, join

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
import plotly.express as px

from IPython.display import display_html

from matplotlib_venn import venn3
from Project.Utils.norm import norm
from Project.Utils.max_corr import max_corr

from Project.Utils.visualize import search, searchTimeSeries, get_years

import nbimporter
import Notebook_Machine_Learning as Nbook_machine

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_path = os.getcwd() + '/Databases/'
output_path = os.getcwd() + '/Output/'
country_path = output_path + '/Country/'
region_path = output_path + '/Region/'
cluster_path = output_path + '/Cluster/'

In [ ]:
file_regions = 'AuxiliarData/world-regions-mod.csv'

file_gold = 'GoldDataframe.csv'
file_corr_pearson = 'Corr_DF_Pearson.csv'
file_corr_spearman = 'Corr_DF_Spearman.csv'
file_agg_region = 'AggregatedRegion_DataFrame.csv'
file_agg_world = 'AggregatedWorld_DataFrame.csv'
file_norm_df = 'Norm_DF.csv'
file_shifted_corr_country = 'Shifted_Corr_Country.csv'
file_shifted_corr_region = 'Shifted_Corr_Region.csv'

In [ ]:
col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'
col_cluster = 'Cluster'
col_shift = 'Shift'

col_1comp = '1st_component'
col_2comp = '2nd_component'

In [ ]:
# READ GOLD DATAFRME
# Read Golden Dataframe and initialize the variables that depend on it.

df_gold = pd.read_csv(output_path + file_gold, on_bad_lines = 'warn', index_col = ['Region', 'Country', 'Year'])

# List of countries and list of regions.
country_list = list(np.sort(df_gold.index.get_level_values(col_country).unique()))
region_list = list(np.sort(df_gold.index.get_level_values(col_region).unique()))

mode_dict = {}
mode_dict[col_country] = country_list
mode_dict[col_region] = region_list
mode_list = [col_country, col_region]

# Range of years.
min_year = df_gold.index.get_level_values(col_year).min()
max_year = df_gold.index.get_level_values(col_year).max()

# List of all indicators, except for the GDP.
indicators_list = df_gold.columns.tolist()
indicators_list.remove(col_gdp)
indicators_list.sort()

In [ ]:
df_regions = pd.read_csv(data_path + file_regions, index_col = [col_country, col_region])
df_corr_pearson = pd.read_csv(output_path + file_corr_pearson, index_col = [col_country])
df_corr_spearman = pd.read_csv(output_path + file_corr_spearman, index_col = [col_country])

median_corr_df_pearson_region = pd.merge(df_corr_pearson, df_regions, how = 'inner', left_index = True, right_index = True).groupby(level = col_region).median()
median_corr_df_spearman_region = pd.merge(df_corr_spearman, df_regions, how = 'inner', left_index = True, right_index = True).groupby(level = col_region).median()
median_corr_df_pearson = pd.merge(df_corr_pearson, df_regions, how = 'inner', left_index = True, right_index = True).median().rename('GDP Pearson Corr')
median_corr_df_spearman = pd.merge(df_corr_spearman, df_regions, how = 'inner', left_index = True, right_index = True).median().rename('GDP Spearman Corr')

median_corr_df = pd.concat([median_corr_df_pearson, median_corr_df_spearman], axis = 1)

shift_dict = {}
shift_dict[col_country] = pd.read_csv(output_path + file_shifted_corr_country, index_col = [col_country, col_shift])
shift_dict[col_region] = pd.read_csv(output_path + file_shifted_corr_region, index_col = [col_region, col_shift])

In [ ]:
# READ CLUSTERS
# Read both Dataframes for each indicators group: the one with the indicator values, and the one with the components.

ind_dict = {}
cluster_dict = {}
cluster_list = []
comp_dict = {}

for element in listdir(cluster_path):
    url = join(cluster_path, element)
    if isfile(url) and url.endswith('_Comp.csv'):
        ind = element.removesuffix('_Comp.csv')
        comp_df = pd.read_csv(url, on_bad_lines = 'warn', index_col = [col_cluster, col_country])
        comp_dict[ind] = comp_df
        cluster_number_df = comp_df.drop(columns = comp_df.columns).reset_index(col_cluster, drop = False).rename({col_cluster: ind}, axis = 1)
        cluster_list.append(cluster_number_df)

    elif isfile(url) and url.endswith('.csv'):
        ind = element.removesuffix('.csv')
        ind_df = pd.read_csv(url, on_bad_lines = 'warn', index_col = [col_cluster, col_country])
        cluster_dict[ind] = ind_df.reset_index(col_cluster, drop = False)
        ind_dict[ind] = np.sort(ind_df.columns)

cluster_df = pd.concat(cluster_list, axis = 1)
cluster_df = cluster_df.fillna(-1).astype(int)

In [ ]:
plot_time_series_out = widgets.Output(layout={'border': '1px solid black'})
plot_cluster_map_out = widgets.Output(layout={'border': '1px solid black'})
table_cluster_out = widgets.Output(layout={'border': '1px solid black'})
venn_group_out = widgets.Output(layout={'border': '1px solid black'})
globe_corr_out = widgets.Output(layout={'border': '1px solid black'})
table_time_window_out = widgets.Output(layout={'border': '1px solid black', 'width': '1120px'})
table_high_range_out = widgets.Output(layout={'border': '1px solid black', 'width': '750px'})
table_corr_country_out = widgets.Output(layout={'border': '1px solid black'})
table_corr_med_out = widgets.Output(layout={'border': '1px solid black'})
table_max_shift_out = widgets.Output(layout={'border': '1px solid black'})
plot_prediction_out = widgets.Output(layout={'border': '1px solid black'})

widgets_general_out = widgets.Output(layout={'border': '1px solid black'})
widgets_time_out = widgets.Output(layout={'border': '1px solid black'})
widgets_cluster_out = widgets.Output(layout={'border': '1px solid black'})

widgets_out = widgets.Output(layout={'border': '1px solid black'})

In [ ]:
@plot_cluster_map_out.capture(clear_output=True, wait=True)
def plot_cluster_map (df: pd.DataFrame, group_name: str = 'All indicators', *, col_1comp: str = '1st_component', col_2comp: str = '2nd_component'):
    if df is None: return None
    fig = px.scatter(df, x = col_1comp, y = col_2comp, text = df.index, size_max=100, color=col_cluster, category_orders={col_cluster: np.sort(df.loc[:, col_cluster].unique())})
    fig.update_layout(title_text = group_name, title_x=0.5)
    fig.update_traces(textposition = 'top center')
    fig.show()

@table_cluster_out.capture(clear_output=True, wait=True)
def table_cluster (df: pd.DataFrame, group_name: str = 'All indicators', country: str = '', cluster_number: int = 0):
    if df is None: return None
    try:
        df_s = df.style
        df_s.apply_index(lambda i: ['background-color: #aadfff; font-weight: 500' if c == country else '' for c in i], axis = 0)
        df_s.apply(lambda row: ['background-color: #ccebff;' if row.name == country else '' for cell in row], axis = 1)
        df_s.set_table_styles([{'selector': 'td:hover', 'props': [('background-color', '#ddfdff')]}])
        tt = {}
        for col in df.columns:
            tt[col] = 'Column median: ' + str(df.loc[:, col].median())
        df_s.set_tooltips(pd.DataFrame(tt, index = df.index))

        # Display a short descriptive title and the Dataframe.
        df_s.set_caption(country + ' belongs to Cluster ' + str(cluster_number) + '. This Cluster contains a total of ' + str(df.shape[0]) + ' countries. The correlation values for these countries are the following:')
        #display(country + ' belongs to Cluster ' + str(cluster_number) + '. This Cluster contains a total of ' + str(df.shape[0]) + ' countries.')
        display(df_s)
    except Exception:
        return print('No indicators available for this country.')

@venn_group_out.capture(clear_output=True, wait=True)
def venn_group(cluster_df, country):
    try:
        _, econ_ind, eq_ind, socdem_ind = ind_dict.keys()       # NON ROBUST STATEMENT
        _, set_econ, set_eq, set_socdem = (set(cluster_df.loc[lambda df: df[ind] == df.loc[country, ind], ind].index.to_list()) for ind in ind_dict.keys())
        venn = venn3([set_econ, set_socdem, set_eq], (econ_ind, socdem_ind, eq_ind))
        plt.rcParams["figure.figsize"] = (12, 12)
        venn.get_label_by_id('100').set_text('\n'.join(set_econ - set_socdem - set_eq)) # Only econ
        venn.get_label_by_id('010').set_text('\n'.join(set_socdem - set_econ - set_eq)) # Only socdem
        venn.get_label_by_id('001').set_text('\n'.join(set_eq - set_econ - set_socdem)) # Only eq

        # The three pair-intersections is guaranteed only if there is an intersection of the three groups.
        if len(set_econ & set_socdem & set_eq):
            venn.get_label_by_id('111').set_text('\n'.join(set_econ & set_socdem & set_eq))
            venn.get_label_by_id('110').set_text('\n'.join(set_econ & set_socdem - set_eq))
            venn.get_label_by_id('101').set_text('\n'.join(set_econ & set_eq - set_socdem))
            venn.get_label_by_id('011').set_text('\n'.join(set_socdem & set_eq - set_econ))
        else:
            # If no center, check the intersections that do exist.
            if len(set_econ & set_socdem - set_eq):
                venn.get_label_by_id('110').set_text('\n'.join(set_econ & set_socdem - set_eq))
            if len(set_econ & set_eq - set_socdem):
                venn.get_label_by_id('101').set_text('\n'.join(set_econ & set_eq - set_socdem))
            if len(set_socdem & set_eq - set_econ):
                venn.get_label_by_id('011').set_text('\n'.join(set_socdem & set_eq - set_econ))
 
        plt.show()
    except:
        print('No indicators available for this country.')

@plot_time_series_out.capture(clear_output=True, wait=True)
def plot_time_series(df: pd.DataFrame, mode: str, zone: str, indicator: str, years: tuple, shift: int):
    data_s = df.loc[df.index.get_level_values(mode) == zone, [col_gdp, indicator]].groupby(level = col_year).median()

    min_year = years[0]
    max_year = years[1]

    min_year_gdp = min_year + max(shift, 0)
    max_year_gdp = max_year + min(shift, 0)

    min_year_ind = min_year - min(shift, 0)
    max_year_ind = max_year - max(shift, 0)

    norm_gdp = norm(data_s.loc[min_year_gdp : max_year_gdp, [col_gdp]], None)
    norm_ind = norm(data_s.loc[min_year_ind : max_year_ind, [indicator]], None)

    plt.figure(figsize = (8,8))
    plt.plot(
                #norm_gdp.index.get_level_values(col_year),
                norm_gdp.reset_index(drop = True),
                color = "red", label = col_gdp)
    plt.plot(
                #norm_ind.index.get_level_values(col_year),
                norm_ind.reset_index(drop = True),
                color = "green", label = indicator)
    plt.legend(loc = "lower right")
    plt.title('GDP - Indicator evolution')
    plt.show()

@table_max_shift_out.capture(clear_output=True, wait=True)
def table_max_shift(mode: str, sh_range: tuple, pval: float):    
    # Load the corresponding Dataframe and apply the user-introduced restrictions.
    df = shift_dict[mode] # CHANGE THIS ONE LINE
    min_sh = sh_range[0]
    max_sh = sh_range[1]
    df = df.loc[(min_sh <= df.index.get_level_values(col_shift)) & (df.index.get_level_values(col_shift) <= max_sh)]
    max_corr_df, max_corr_index_df = max_corr(df, mode, pval, raw = False)

    # Apply style and display.
    df_s = max_corr_df.style
    df_s.set_tooltips(max_corr_index_df.applymap(lambda x:
                                                    'No data' if np.isnan(x) else 
                                                    'Shift: ' + str(int(x))                         
                                                )
                    )
    df_s.background_gradient(cmap='RdBu')
    df_s.set_caption('Maximum correlation values (hover to show which shift was applied)')
    display(df_s)

In [ ]:
import itertools
from scipy import stats

def styler_method(df, name, pvalue = None):    
        if pvalue == None:
                pvalue = 0.05
    #try:
        styles = [dict(selector="caption", props=[("background-color", "#98D3FF")])]
        left1 = pd.Series([pvalue], index=['P-value Spearman'])
        left2 = pd.Series([-1], index=['GDP Spearman Corr'])
        left3 = pd.Series([0], index=['GDP Spearman Corr'])
        dfs = df.style.highlight_between(left = left1, right = 1.5, axis = 1, props='color:white; background-color:red;')\
                .highlight_between(left = left2, right = 1.5, axis = 1, props='color:white; background-color:#929bfc;')\
                .highlight_between(left = left3, right = 1.5, axis = 1, props='color:white; background-color:#b3b9ff;')\
                .format('{:,.4f}', subset = ['GDP Spearman Corr'])\
                .format('{:,.12f}', subset = ['P-value Spearman']) \
                .set_caption(name).set_table_styles(styles)\
                .set_table_attributes("style='display:inline'")
    #except:
    #    dfs = 'No indicators have been found for the window dataframe in this range.'
        
        return dfs

def init_highest_table(indicators):    
    df_highest = pd.DataFrame(columns={"Indicator"})
    df_highest["Indicator"] = indicators
    df_highest["Year range"] = 0
    df_highest["Highest positive corr"] = 0
    df_highest["Year range "] = 0
    df_highest["Highest negative corr"] = 0
    df_highest.set_index("Indicator", inplace= True)

    return df_highest

def generate_years_combinations(min_diff: int, min, max):
    iterable = list(range(min, max + 1))
    iterable = list(itertools.combinations(iterable, 2))

    #The year length must be higher than 5. All entries with a lower range are deleted.
    for years in iterable.copy():
        if (years[1] - years[0]) < min_diff:
            iterable.remove(years)
    return iterable

In [ ]:
@globe_corr_out.capture(clear_output=True, wait=True)
def globe_corr(df_corr, ind):
    N = 10
    fig = px.choropleth(df_corr, locations = df_corr.index, locationmode='country names', 
                        color = ind, projection="natural earth",
                        color_continuous_scale='RdBu',
                        width = 700, height=500)

    pos_corr = df_corr.drop(df_corr.columns.difference([ind]), axis = 1).sort_values(by = ind, axis = 0, ascending = False).head(n = N)
    neg_corr = df_corr.drop(df_corr.columns.difference([ind]), axis = 1).sort_values(by = ind, axis = 0, ascending = True).head(n = N)

    pos_corr = pos_corr.loc[pos_corr[ind] > 0]
    neg_corr = neg_corr.loc[neg_corr[ind] < 0]

    fig.update_layout(title_text = 'Correlation to GDP')
    fig.update(layout_coloraxis_showscale=True)
    fig.show()

@table_time_window_out.capture(clear_output=True, wait=True)
def table_time_window(df, mode, zone, years, threshold, pval):
    df_zone = df.loc[df.index.get_level_values(mode) == zone]
    #Load the selected year range and the global range.
    df_time = searchTimeSeries(threshold, years[0], years[1], True, df_zone)
    df_global = searchTimeSeries(threshold, years[0], years[1], False, df_zone)

    # Display Data
    if years[0] > years[1]: return print("Please, select a valid range of years.")
 
    space = "\xa0" * 5
    try:
        df_time = styler_method(df_time, str(years[0]) + '-' + str(years[1]), pval)._repr_html_()
    except Exception as e: 
        df_time = 'No indicators available for the selected parameters'
    
    try:
        df_global = styler_method(df_global, '2000-2020 (Original)', pval)._repr_html_()
    except: 
        df_global = 'No indicators available for the selected parameters'
    
    display_html(df_time + space  + df_global, raw=True)

@table_high_range_out.capture(clear_output=True)
def table_high_range(df, mode, zone, pval):
    if mode == 'Country':
        min_diff = 5
        
    elif mode == 'Region':
        min_diff = 2
    
    df_zone = df.loc[df.index.get_level_values(mode) == zone] #.drop(columns = 'GDP')
    indicators = df.columns
    df_highest = init_highest_table(indicators)

    i = 0
    computing_text = "Loading "
    print(computing_text, end="\r")
    
    #For all the combination of years...
    for years in generate_years_combinations(min_diff, min_year, max_year):
        i = (i + 1) % 50
        print (computing_text + "".join(["." for _ in range(i)]), end="\r")
        
        df_aux = searchTimeSeries(0, years[0], years[1], True, df_zone)
        #Delete indicators which are not available that year
        indicators_inter = list(set(indicators) & set(list(df_aux.index)))

        #For all the indicators availble that year....
        for indicator in indicators_inter:
            #Algorithm to search for the highest value
            indicator_corr_pos_last = df_highest[df_highest.index.get_level_values(0) == indicator]["Highest positive corr"][0]
            indicator_corr_neg_last = df_highest[df_highest.index.get_level_values(0) == indicator]["Highest negative corr"][0]
            
          
            indicator_corr_aux = df_aux[df_aux.index.get_level_values(0) == indicator]["GDP Spearman Corr"][0]
            indicator_p_value_aux = df_aux[df_aux.index.get_level_values(0) == indicator]["P-value Spearman"][0]

            if indicator_corr_aux != np.NaN and indicator_p_value_aux < pval:
                if indicator_corr_pos_last < indicator_corr_aux and indicator_corr_aux > 0:
                    df_highest.at[indicator, "Year range"] = str(years[0]) + '-' + str(years[1])
                    df_highest.at[indicator, "Highest positive corr"] = indicator_corr_aux
                elif indicator_corr_neg_last > indicator_corr_aux and indicator_corr_aux < 0:
                    df_highest.at[indicator, "Year range "] = str(years[0]) + '-' + str(years[1])
                    df_highest.at[indicator, "Highest negative corr"] = indicator_corr_aux

    df_highest = df_highest.replace(0, np.NaN).dropna(axis=0, how='all').fillna("-")

    print("                                                                                    ", end="\r")
    display(df_highest)

@table_corr_country_out.capture(clear_output=True, wait=True)
def table_corr_country(mode, zone, threshold, pval):
#     print(mode, zone)
    try:
        df = search(threshold, mode, zone)
        if df.empty:
            return print("No indicators have been found.")

        left1 = pd.Series([pval, pval], index=['P-value Pearson', 'P-value Spearman'])
        left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
        left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])

        df = df.style.highlight_between(left = left1, right = 1.5, axis = 1, props='color:white; background-color:red;')\
                    .highlight_between(left = left2, right = 1.5, axis = 1, props='color:white; background-color:#929bfc;')\
                    .highlight_between(left = left3, right = 1.5, axis = 1, props='color:white; background-color:#b3b9ff;')\
                    .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
                    .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman'])\
                    .set_caption(zone + ' correlations')          
        display(df)
    except:
        return print('Loading...')

@table_corr_med_out.capture(clear_output=True, wait=True)
def table_corr_med(threshold):
    df = median_corr_df
    df = df.loc[(abs(df['GDP Pearson Corr']) >= threshold) & (abs(df['GDP Spearman Corr']) >= threshold)]
    if df.empty:
        return print("No indicators have been found.")

    df = df.sort_values(by = df.columns[0], ascending = False, key = lambda row: df.sum(axis = 1))

    left1 = pd.Series([-1, -1], index = ['GDP Pearson Corr', 'GDP Spearman Corr'])
    left2 = pd.Series([0, 0], index = ['GDP Pearson Corr', 'GDP Spearman Corr'])

    df = df.style.highlight_between(left = left1, right = 1.5, axis = 1, props = 'color:white; background-color:#929bfc;')\
                 .highlight_between(left = left2, right = 1.5, axis = 1, props = 'color:white; background-color:#b3b9ff;')\
                 .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
                 .set_caption('World Median Correlation')                
    display(df)

@plot_prediction_out.capture(clear_output=True, wait=True)
def plot_prediction(mode, zone):
    if mode != col_country:
        display('')
    else:
        df_gold_aux = pd.read_csv(output_path + file_gold, index_col = ['Country', 'Year', 'Region'])
        df, country_cluster_list = Nbook_machine.get_cluster_countries(zone, df_gold_aux, cluster_dict['All indicators'].reset_index())
        model, score, feature_df = Nbook_machine.machine_algorithm(df, country_cluster_list)
        Nbook_machine.plot_machine(model, score, feature_df, zone, df_gold_aux)

In [ ]:
dropdown_mode = widgets.Dropdown(
    options = mode_list,
    description = 'Select: '
)

dropdown_zone = widgets.Dropdown(
    options = mode_dict[dropdown_mode.value],
    description = 'Show: '
)

dropdown_group = widgets.Dropdown(
    options = ind_dict.keys(),
    description = 'Group: '
)

dropdown_ind = widgets.Dropdown(
    options = indicators_list,
    description = 'Indicator: '
)

intrangeslider_years = widgets.IntRangeSlider(
    value = [min_year, max_year],
    min = min_year,
    max = max_year,
    step = 1,
    description = 'Years: ',
)

intrangeslider_shift_range = widgets.IntRangeSlider(
    value = (0, (max_year - min_year - 1)),
    min = min_year - max_year + 1,
    max = max_year - min_year - 1,
    step = 1,
    description = 'Shifts Range: '
)

intslider_shift = widgets.IntSlider(
    value = 0,
    min = min_year - max_year + 1,
    max = max_year - min_year - 1,
    step = 1,
    description = 'Shifts: '
)

floatslider_threshold = widgets.FloatSlider(
    value = 0.7,
    min = 0.0,
    max = 1.0,
    step = 0.05,
    description = 'Threshold: '
)

floatslider_confidence = widgets.FloatSlider(
    value = 0.95,
    min = 0.0,
    max = 1.0,
    step = 0.05,
    description = 'Confidence: '
)

loading_label = widgets.Label(value = '')

zone_title = widgets.Label(value = dropdown_zone.value)

In [ ]:
class WidgetStatus:
    def __init__(self):
        
        self.mode = None
        self.zone = None
        self.ind = None
        self.ind_group = None #dropdown_group.value
        self.years = list(intrangeslider_years.value)
        self.sh_range = None #list(intrangeslider_shift_range.value)
        self.shift = intslider_shift.value
        self.threshold = None
        self.pval = None

#         self.mode = dropdown_mode.value
#         self.zone = dropdown_zone.value
#         self.ind = dropdown_ind.value
#         self.ind_group = dropdown_group.value
#         self.years = list(intrangeslider_years.value)
#         self.shift = intslider_shift.value
#         self.threshold = floatslider_threshold.value
#         self.pval = 1 - floatslider_confidence.value

        self.df_gold = pd.read_csv(output_path + file_gold, index_col = [col_country, col_year, col_region])
        self.corr_df = pd.read_csv(output_path + file_corr_spearman, index_col = [col_country])
        self.norm_df = pd.read_csv(output_path + file_norm_df, index_col = [col_country, col_region, col_year])
        self.cluster_df = cluster_df
        self.comp_df = None #comp_dict[self.ind_group].reset_index(col_cluster, drop = False)
        self.ind_df = None

        self.clus_num = None #cluster_df.loc[cluster_df.index.get_level_values(col_country) == dropdown_zone.value, dropdown_group.value].item()

    def _changed_zone(self):
        # General
        loading_label.value = 'Loading general tab'
        table_corr_country(self.mode, self.zone, self.threshold, self.pval)
        table_corr_med(self.threshold)
        plot_prediction(self.mode, self.zone)

        # Time
        loading_label.value = 'Loading time tab'
        table_time_window(self.df_gold, self.mode, self.zone, self.years, self.threshold, self.pval)
        plot_time_series(self.df_gold, self.mode, self.zone, self.ind, self.years, self.shift)
        table_high_range(self.df_gold, self.mode, self.zone, self.pval)

        # Clusters
        loading_label.value = 'Loading cluster tab'
        venn_group (self.cluster_df, self.zone)
        #table_cluster (self.ind_df, self.ind_group, self.zone)
        #plot_cluster_map (self.comp_df, self.ind_group)

        loading_label.value = ''
    
    def _changed_indicator(self):
        # General
        loading_label.value = 'Loading general tab'
        globe_corr(self.corr_df, self.ind)

        # Time
        loading_label.value = 'Loading time tab'
        table_time_window(self.df_gold, self.mode, self.zone, self.years, self.threshold, self.pval)
        plot_time_series(self.df_gold, self.mode, self.zone, self.ind, self.years, self.shift)
        loading_label.value = ''

    def _changed_group(self):
        # Clusters
        loading_label.value = 'Loading cluster tab'
        #venn_group (self.cluster_df, self.zone)
        #table_cluster (self.ind_df, self.ind_group, self.zone)
        self.comp_df = comp_dict[self.ind_group].reset_index(col_cluster, drop = False)
        plot_cluster_map (self.comp_df, self.ind_group)

        loading_label.value = ''

    def _changed_years(self):
        # Time
        loading_label.value = 'Loading time tab'
        table_time_window(self.df_gold, self.mode, self.zone, self.years, self.threshold, self.pval)
        plot_time_series(self.df_gold, self.mode, self.zone, self.ind, self.years, self.shift)

        loading_label.value = ''
    
    def _changed_sh_range(self):
        # Time
        loading_label.value = 'Loading time tab'
        table_max_shift(self.mode, self.sh_range, self.pval)

        loading_label.value = ''

    def _changed_shift(self):
        # Time
        loading_label.value = 'Loading time tab'
        plot_time_series(self.df_gold, self.mode, self.zone, self.ind, self.years, self.shift)

        loading_label.value = ''

    def _changed_threshold(self):
        # General
        loading_label.value = 'Loading general tab'
        table_corr_country(self.mode, self.zone, self.threshold, self.pval)
        table_corr_med(self.threshold)

        # Time
        loading_label.value = 'Loading time tab'
        table_time_window(self.df_gold, self.mode, self.zone, self.years, self.threshold, self.pval)

        loading_label.value = ''

    def _changed_confidence(self):
        # General
        loading_label.value = 'Loading general tab'
        table_corr_country(self.mode, self.zone, self.threshold, self.pval)

        # Time
        loading_label.value = 'Loading time tab'
        table_time_window(self.df_gold, self.mode, self.zone, self.years, self.threshold, self.pval)
        table_high_range(self.df_gold, self.mode, self.zone, self.pval)

        loading_label.value = ''
    
    def _changed_cluster(self):
        # Clusters
        loading_label.value = 'Loading cluster tab'
        table_cluster (self.ind_df, self.ind_group, self.zone)

        loading_label.value = ''

    def update(self, mode = None, zone = None, ind = None, group = None, year_min = None, year_max = None, sh_range = None, shift = None, threshold = None, confidence = None):
        if mode and mode != self.mode:
            self.mode = mode
            dropdown_zone.options = mode_dict[mode]
            dropdown_zone.value = mode_dict[mode][0]
        
        if zone and zone != self.zone:
            self.zone = zone
            self.clus_num = None
            self._changed_zone()
    
        if ind and ind != self.ind:
            self.ind = ind
            self._changed_indicator()
        
        if group and group != self.ind_group:
            self.ind_group = group
            self.comp_df = comp_dict[self.ind_group].reset_index(col_cluster, drop = False)
            self.clus_num = None
            self._changed_group()
        
        if year_min and year_min != self.years[0]:
            self.years[0] = year_min
            self._changed_years()

        if year_max and year_max != self.years[1]:
            self.years[1] = year_max
            self._changed_years()
        
        if sh_range and sh_range != self.sh_range:
            self.sh_range = sh_range
            self._changed_sh_range()

        if shift and shift != self.shift:
            self.shift = shift
            self._changed_shift()
        
        if threshold and threshold != self.threshold:
            self.threshold = threshold
            self._changed_threshold()

        if confidence and (1 - confidence) != self.pval:
            self.pval = 1 - confidence
            self._changed_confidence()

        if (self.clus_num is None) and group and (self.mode == col_country):
                self.clus_num = self.cluster_df.loc[self.cluster_df.index.get_level_values(col_country) == self.zone, self.ind_group].item()
                self.ind_df = cluster_dict[self.ind_group].loc[cluster_dict[self.ind_group][col_cluster] == self.clus_num].drop(col_cluster, axis = 'columns')
                self._changed_cluster()
        
status = WidgetStatus()

In [ ]:
# GENERAL LAYOUT:
general_upper = widgets.HBox(children = [widgets_general_out, plot_time_series_out, globe_corr_out])
general_lower = widgets.HBox(children = [table_corr_country_out, table_corr_med_out, plot_prediction_out])
general_tab = widgets.VBox(children = [general_upper, general_lower])

# TIME LAYOUT
time_upper = widgets.HBox(children = [widgets_time_out, plot_time_series_out, table_time_window_out, table_high_range_out])
time_lower = widgets.HBox(children = [table_max_shift_out])
time_tab = widgets.VBox(children = [time_upper, time_lower])

# CLUSTER LAYOUT
cluster_upper = widgets.HBox(children = [widgets_cluster_out, venn_group_out, plot_cluster_map_out])
cluster_lower = widgets.HBox(children = [table_cluster_out])
cluster_tab = widgets.VBox(children = [cluster_upper, cluster_lower])

tab = widgets.Tab(children = [general_tab, time_tab, cluster_tab])

tab.set_title(0, 'General')
tab.set_title(1, 'Time Series')
tab.set_title(2, 'Cluster')

In [ ]:
def show_general(mode, zone, ind, threshold, confidence):
    status.update(mode = mode, zone = zone, ind = ind, threshold = threshold, confidence = confidence)
        
def show_time(mode, zone, ind, years, sh_range, shift):
    status.update(mode = mode, zone = zone, ind = ind, year_min = years[0], year_max = years[1], sh_range = sh_range, shift = shift)

def show_cluster(zone, group):
    status.update(zone = zone, group = group)

In [ ]:
display(tab)

In [ ]:
with widgets_general_out:
    widgets_general_out.clear_output()
    widgets.interact(show_general, mode = dropdown_mode, zone = dropdown_zone, ind = dropdown_ind,
                threshold = floatslider_threshold, confidence = floatslider_confidence)
    display(loading_label)

with widgets_time_out:
    widgets_time_out.clear_output()
    widgets.interact(show_time, mode = dropdown_mode, zone = dropdown_zone, ind = dropdown_ind,
                years = intrangeslider_years, sh_range = intrangeslider_shift_range, shift = intslider_shift)
    display(loading_label)

with widgets_cluster_out:
    widgets_cluster_out.clear_output()
    widgets.interact(show_cluster, zone = dropdown_zone, group = dropdown_group)
    display(loading_label)